In [34]:
from IPython.display import display
import ipywidgets as widgets

from pulp import LpProblem, LpVariable, lpSum, LpMaximize

# Auswahl

In [98]:
# Example project data (project_id, priority, crane_capcity_required, driver_skill_required)
projects = {
    'P1': (1, 2, 1, 'Sunny', 'Type1', 5),  # Project P1 requires 5 hours
    'P2': (2, 1, 2, 'Rainy', 'Type2', 7),  # Project P2 requires 7 hours
    'P3': (3, 3, 2, 'Partly Cloudy','Type2', 3)
}

# Example crane data (crane_id, type, capacity)
cranes = {
    'C1': ('Type1', 5),
    'C2': ('Type2', 6),
    'C3': ('Type1', 7),
    # Add more cranes as needed
}

# Example driver data (driver_id, skill_level)
drivers = {
    'D1': 3,
    'D2': 2,
    'D3': 4,
    # Add more drivers as needed
}


In [84]:
# Create an Output widget for displaying the saved items
output = widgets.Output()

# Create a SelectMultiple widget for each dictionary
select_dict1 = widgets.SelectMultiple(
    options=projects.keys(),
    description='Aufträge',
)
select_dict2 = widgets.SelectMultiple(
    options=cranes.keys(),
    description='Kräne',
)
select_dict3 = widgets.SelectMultiple(
    options=drivers.keys(),
    description='Fahrer',
)

# Display the SelectMultiple widgets
#display(select_dict1, select_dict2, select_dict3)

# Initialize variables to store selected keys
selected_keys_dict1 = []
selected_keys_dict2 = []
selected_keys_dict3 = []
# Function to handle selection changes
def handle_selection_change(change):
    global selected_keys_dict1, selected_keys_dict2, selected_keys_dict3
    selected_keys_dict1 = list(select_dict1.value)
    selected_keys_dict2 = list(select_dict2.value)
    selected_keys_dict3 = list(select_dict3.value)

# Attach the function to selection changes
select_dict1.observe(handle_selection_change, names='value')
select_dict2.observe(handle_selection_change, names='value')
select_dict3.observe(handle_selection_change, names='value')



# Function to save selected dictionaries
def save_selected_dictionaries(btn):

    selected_dict1 = {key: dict1[key] for key in selected_keys_dict1}
    selected_dict2 = {key: dict2[key] for key in selected_keys_dict2}
    selected_dict3 = {key: dict3[key] for key in selected_keys_dict3}
    #select_box = widgets.VBox([selected_dict1, selected_dict2, selected_dict3])
    # Update the content of the output widget with selected keys
    #output.clear_output()
    with output:
        clear_output(wait=True)
        print("Selected Keys:")
        print(f"Dictionary 1: {selected_keys_dict1}")
        print(f"Dictionary 2: {selected_keys_dict2}")
        print(f"Dictionary 3: {selected_keys_dict3}")



#display(widgets.HBox([select_box, output, save_button]))


# Create a button to trigger saving
save_button = widgets.Button(description="Auswahl speichern!")
save_button.on_click(lambda btn: save_selected_dictionaries())
# Display the widgets
box2 = widgets.VBox([output])

display(select_dict1, select_dict2, select_dict3, save_button)
#item_box = widgets.VBox([widgets.Label(item) for item in selected_keys_dict1])
display(box2)


SelectMultiple(description='Aufträge', options=('P1', 'P2', 'P3'), value=())

SelectMultiple(description='Kräne', options=('C1', 'C2', 'C3'), value=())

SelectMultiple(description='Fahrer', options=('D1', 'D2', 'D3'), value=())

Button(description='Auswahl speichern!', style=ButtonStyle())

In [101]:
# Using dictionary comprehension to select entries with matching keys
projects_selected = {key: value for key, value in projects.items() if key in selected_keys_dict1}
cranes_selected = {key: value for key, value in cranes.items() if key in selected_keys_dict2}
drivers_selected = {key: value for key, value in drivers.items() if key in selected_keys_dict3}
print(projects_selected)
print(projects)
print(cranes_selected)

{'P1': (1, 2, 1, 'Sunny', 'Type1', 5), 'P2': (2, 1, 2, 'Rainy', 'Type2', 7), 'P3': (3, 3, 2, 'Partly Cloudy', 'Type2', 3)}
{'P1': (1, 2, 1, 'Sunny', 'Type1', 5), 'P2': (2, 1, 2, 'Rainy', 'Type2', 7), 'P3': (3, 3, 2, 'Partly Cloudy', 'Type2', 3)}
{'C1': ('Type1', 5), 'C2': ('Type2', 6), 'C3': ('Type1', 7)}


In [102]:
output = widgets.Output()
# Create a VBox (Vertical Box) to hold the items
project_box = widgets.HBox([widgets.Label(item) for item in selected_keys_dict1], description="Auswahl Aufträge")
crane_box = widgets.HBox([widgets.Label(item) for item in selected_keys_dict2], description="Auswahl Kräne")
driver_box = widgets.HBox([widgets.Label(item) for item in selected_keys_dict3], description="Auswahl Faherer")
#box2 = widgets.VBox([output])
#box2 = widgets.VBox([output])
# Display the VBox
display(project_box, crane_box, driver_box)


# Model 
- Intelligente Linear Programming Optmierung

In [103]:
# Create the optimization model
model = LpProblem("Resource_Allocation", LpMaximize)
time_periods = range(1, 25)


In [104]:
x = {(p, c, d, t): LpVariable(f'x_{p}_{c}_{d}_{t}', cat='Binary') for p in projects_selected for c in cranes_selected for d in drivers_selected for t in time_periods}


# Zielfunktion
- Priorität-optmierte Zuordnung

In [105]:
# Objective function: Maximize total project priorities considering project order
model += lpSum(projects_selected[p][0] * x[p, c, d, t] for p in projects_selected for c in cranes_selected for d in drivers_selected for t in time_periods), "Total_Priority_With_Time"


# Zwangsbedingungen
- ein Kran und Fahrer pro Auftrag
- der Fahrer muss die Skills für einen gewissen Kran haben
- Die Betriebszeiten sind von 6-20 Uhr, alle Projecte bis Dienstschluss abschließen

In [106]:

# Constraint: Each project gets one crane and one driver
for p in projects_selected:
   model += lpSum(x[p, c, d,t] for c in cranes_selected for d in drivers_selected for t in time_periods) == 1, f'Project_{p}_Allocation'


# Constraint: Each crane can be allocated to only one project at a time
for c in cranes_selected:
    for t in time_periods:
        max_allocation_time = min(t + cranes_selected[c][1], max(time_periods))
        model += (
            lpSum(x[p, c, d, t_prime] for p in projects for d in drivers for t_prime in range(t, max_allocation_time)) <= 1,
            f'Crane_Allocation_Constraint_{c}_{t}'
        )

# Constraint: Allocate crane only if project starts after 8:00 and finishes before 20:00 (8:00 PM)
for p in projects_selected:
    for c in cranes_selected:
        for d in drivers_selected:
            for t in time_periods:
                # Calculate the project's end time based on its start time and duration
                end_time = t + projects_selected[p][5] - 1
                start_time = 6
                #print(projects[p][5])
               
                # Check if the project finishes before 20:00
                if (end_time > 20) or (t < start_time):
                     #print(end_time)
                     model += x[p, c, d, t] == 0, f'Crane_Allocation_Time_Constraint_{p}_{c}_{d}_{t}'

# Constraint: Skill level constraint for drivers
for d in drivers_selected:
    model += lpSum(projects_selected[p][2] * x[p, c, d, t] for p in projects_selected for c in cranes_selected for t in time_periods) <= drivers_selected[d], f'Driver_{d}_Skill'

              

# Die optimale Lösung

In [107]:
result = model.solve()
print(result)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/michiundslavki/miniconda3/envs/Resource_Allo/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/vw/4d6l2vcs2_d7xcp6s_tqf9mw0000gn/T/f4100174318f4a5a84e49e6796de5a64-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/vw/4d6l2vcs2_d7xcp6s_tqf9mw0000gn/T/f4100174318f4a5a84e49e6796de5a64-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 434 COLUMNS
At line 7338 RHS
At line 7768 BOUNDS
At line 8417 ENDATA
Problem MODEL has 429 rows, 648 columns and 4959 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6 - 0.00 seconds
Cgl0002I 351 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8 strengthene

In [108]:
print("Hallo ich bin dein CraneAI Dispatch Manager") 

if  result == -1: 
    print("Die Aufträge übersteigen deine verfügbaren Kapazitäten")
elif result == 1: 
    print("Das ist mein Vorschlag für deine Auswahl:")
else: 
    ("Es ist was bei der Optimierung schiefgelaufen, bitte nochmals versuchen")
for p in projects_selected:
    for c in cranes_selected:
        for d in drivers_selected:
            for t in time_periods:
                if x[p, c, d,t].varValue == 1:
                 print(f" Auftrag {p} zu Kran {c} mit Fahrer {d} zur Uhrzeit {t} zuordnen")


Hallo ich bin dein CraneAI Dispatch Manager
Das ist mein Vorschlag für deine Auswahl:
 Auftrag P1 zu Kran C3 mit Fahrer D3 zur Uhrzeit 8 zuordnen
 Auftrag P2 zu Kran C2 mit Fahrer D3 zur Uhrzeit 8 zuordnen
 Auftrag P3 zu Kran C1 mit Fahrer D2 zur Uhrzeit 14 zuordnen


# User Interface

In [4]:
import ipywidgets as widgets
from IPython.display import display

# Define your dictionaries
dict1 = {'Option 1': 1, 'Option 2': 2, 'Option 3': 3}
dict2 = {'A': 'Apple', 'B': 'Banana', 'C': 'Cherry'}
dict3 = {'Red': 'Color1', 'Green': 'Color2', 'Blue': 'Color3'}

# Create a checkbox widget for each dictionary
checkbox_dict1 = widgets.Checkbox(value=False, description='Dict 1')
checkbox_dict2 = widgets.Checkbox(value=False, description='Dict 2')
checkbox_dict3 = widgets.Checkbox(value=False, description='Dict 3')

# Display the checkboxes
display(checkbox_dict1, checkbox_dict2, checkbox_dict3)

# Function to handle checkbox changes
def handle_checkbox_change(change):
    selected_dicts = []
    if checkbox_dict1.value:
        selected_dicts.append(dict1)
    if checkbox_dict2.value:
        selected_dicts.append(dict2)
    if checkbox_dict3.value:
        selected_dicts.append(dict3)
    
    # Process the selected dictionaries here (e.g., print them)
    print("Selected Dictionaries:")
    for d in selected_dicts:
        print(d)

# Attach the function to checkbox changes
checkbox_dict1.observe(handle_checkbox_change, names='value')
checkbox_dict2.observe(handle_checkbox_change, names='value')
checkbox_dict3.observe(handle_checkbox_change, names='value')


Checkbox(value=False, description='Dict 1')

Checkbox(value=False, description='Dict 2')

Checkbox(value=False, description='Dict 3')

In [1]:
import ipywidgets as widgets
from IPython.display import display

# Define your dictionaries
dict1 = {'Option 1': 1, 'Option 2': 2, 'Option 3': 3}
dict2 = {'A': 'Apple', 'B': 'Banana', 'C': 'Cherry'}
dict3 = {'Red': 'Color1', 'Green': 'Color2', 'Blue': 'Color3'}

# Create a SelectMultiple widget for each dictionary
select_dict1 = widgets.SelectMultiple(
    options=dict1.keys(),
    description='Dict 1',
)
select_dict2 = widgets.SelectMultiple(
    options=dict2.keys(),
    description='Dict 2',
)
select_dict3 = widgets.SelectMultiple(
    options=dict3.keys(),
    description='Dict 3',
)

# Display the SelectMultiple widgets
display(select_dict1, select_dict2, select_dict3)

# Function to handle selection changes
def handle_selection_change(change):
    selected_keys_dict1 = select_dict1.value
    selected_keys_dict2 = select_dict2.value
    selected_keys_dict3 = select_dict3.value
    
    # Process the selected keys here (e.g., print them)
    print("Selected Keys:")
    if selected_keys_dict1:
        print(f'Dict 1: {selected_keys_dict1}')
    if selected_keys_dict2:
        print(f'Dict 2: {selected_keys_dict2}')
    if selected_keys_dict3:
        print(f'Dict 3: {selected_keys_dict3}')

# Attach the function to selection changes
select_dict1.observe(handle_selection_change, names='value')
select_dict2.observe(handle_selection_change, names='value')
select_dict3.observe(handle_selection_change, names='value')


SelectMultiple(description='Dict 1', options=('Option 1', 'Option 2', 'Option 3'), value=())

SelectMultiple(description='Dict 2', options=('A', 'B', 'C'), value=())

SelectMultiple(description='Dict 3', options=('Red', 'Green', 'Blue'), value=())

In [5]:
# Define your dictionaries
dict1 = {'Option 1': 1, 'Option 2': 2, 'Option 3': 3}
dict2 = {'A': 'Apple', 'B': 'Banana', 'C': 'Cherry'}
dict3 = {'Red': 'Color1', 'Green': 'Color2', 'Blue': 'Color3'}

# Create a SelectMultiple widget for each dictionary
select_dict1 = widgets.SelectMultiple(
    options=dict1.keys(),
    description='Dict 1',
)
select_dict2 = widgets.SelectMultiple(
    options=dict2.keys(),
    description='Dict 2',
)
select_dict3 = widgets.SelectMultiple(
    options=dict3.keys(),
    description='Dict 3',
)

# Display the SelectMultiple widgets
display(select_dict1, select_dict2, select_dict3)

# Initialize variables to store selected keys
selected_keys_dict1 = []
selected_keys_dict2 = []
selected_keys_dict3 = []

# Function to handle selection changes
def handle_selection_change(change):
    global selected_keys_dict1, selected_keys_dict2, selected_keys_dict3
    selected_keys_dict1 = list(select_dict1.value)
    selected_keys_dict2 = list(select_dict2.value)
    selected_keys_dict3 = list(select_dict3.value)

# Attach the function to selection changes
select_dict1.observe(handle_selection_change, names='value')
select_dict2.observe(handle_selection_change, names='value')
select_dict3.observe(handle_selection_change, names='value')

# Function to save selected dictionaries
def save_selected_dictionaries():
    selected_dict1 = {key: dict1[key] for key in selected_keys_dict1}
    selected_dict2 = {key: dict2[key] for key in selected_keys_dict2}
    selected_dict3 = {key: dict3[key] for key in selected_keys_dict3}
    
    # You can use, print, or store the selected dictionaries as needed
    print("Selected Dictionaries:")
    if selected_dict1:
        print(f'Dict 1: {selected_dict1}')
    if selected_dict2:
        print(f'Dict 2: {selected_dict2}')
    if selected_dict3:
        print(f'Dict 3: {selected_dict3}')

# Create a button to trigger saving
save_button = widgets.Button(description="Save Selected Dictionaries")
save_button.on_click(lambda btn: save_selected_dictionaries())
display(save_button)

SelectMultiple(description='Dict 1', options=('Option 1', 'Option 2', 'Option 3'), value=())

SelectMultiple(description='Dict 2', options=('A', 'B', 'C'), value=())

SelectMultiple(description='Dict 3', options=('Red', 'Green', 'Blue'), value=())

Button(description='Save Selected Dictionaries', style=ButtonStyle())

In [7]:
print(selected_keys_dict1)

['Option 1', 'Option 2']


# Archive

In [ ]:

# Constraint: Each project gets one crane and one driver
#    model += lpSum(x[p, c, d] for c in cranes for d in drivers) == 1, f'Project_{p}_Allocation'

# Constraint: Capacity constraint for cranes
for c in cranes:
    model += lpSum(projects[p][1] * x[p, c, d] for p in projects for d in drivers ) <= cranes[c][1], f'Crane_{c}_Capacity'

# Constraint: Skill level constraint for drivers
for d in drivers:
    model += lpSum(projects[p][2] * x[p, c, d] for p in projects for c in cranes) <= drivers[d], f'Driver_{d}_Skill'

# Constraint: Each project gets one crane and one driver of the correct type
#for p in projects:
#    for d in drivers:
#        model += (
#            lpSum(x[p, c, d] for c in cranes if cranes[c][0] == projects[p][5]) <= 1,
#            f'Allocation_{p}_{d}_Constraint',
#        )




In [ ]:
# Define your dictionaries
#dict1 = {'Option 1': 1, 'Option 2': 2, 'Option 3': 3}
#dict2 = {'A': 'Apple', 'B': 'Banana', 'C': 'Cherry'}
#dict3 = {'Red': 'Color1', 'Green': 'Color2', 'Blue': 'Color3'}

# Create a SelectMultiple widget for each dictionary
select_dict1 = widgets.SelectMultiple(
    options=projects.keys(),
    description='Aufträge',
)
select_dict2 = widgets.SelectMultiple(
    options=cranes.keys(),
    description='Kräne',
)
select_dict3 = widgets.SelectMultiple(
    options=drivers.keys(),
    description='Fahrer',
)

# Display the SelectMultiple widgets
display(select_dict1, select_dict2, select_dict3)

# Initialize variables to store selected keys
selected_keys_dict1 = []
selected_keys_dict2 = []
selected_keys_dict3 = []

# Function to handle selection changes
def handle_selection_change(change):
    global selected_keys_dict1, selected_keys_dict2, selected_keys_dict3
    selected_keys_dict1 = list(select_dict1.value)
    selected_keys_dict2 = list(select_dict2.value)
    selected_keys_dict3 = list(select_dict3.value)

# Attach the function to selection changes
select_dict1.observe(handle_selection_change, names='value')
select_dict2.observe(handle_selection_change, names='value')
select_dict3.observe(handle_selection_change, names='value')

# Function to save selected dictionaries
def save_selected_dictionaries():
    selected_dict1 = {key: dict1[key] for key in selected_keys_dict1}
    selected_dict2 = {key: dict2[key] for key in selected_keys_dict2}
    selected_dict3 = {key: dict3[key] for key in selected_keys_dict3}
    print(f"Aufträge {selected_keys_dict1}")
    # You can use, print, or store the selected dictionaries as needed
    print("Selected Dictionaries:")


# Create a button to trigger saving
save_button = widgets.Button(description="Auswahl speichern!")
save_button.on_click(lambda btn: save_selected_dictionaries())
display(save_button)


In [ ]:
# Objective function: Maximize total project priorities
model += lpSum(projects_selected[p][0] * x[p, c, d] for p in projects_selected for c in cranes_selected for d in drivers_selected), "Total_Priority"
#model += lpSum(projects[p] * x[p, c, d] for p in projects for c in cranes for d in drivers), "Total_Priority"

In [ ]:
x = {(p, c, d): LpVariable(f'x_{p}_{c}_{d}', cat='Binary') for p in projects_selected for c in cranes_selected for d in drivers_selected}